# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [9]:
from pyspark.ml.feature import VectorAssembler

In [21]:
from pyspark.ml.regression import LinearRegression

In [47]:
from pyspark.ml.feature import StringIndexer

In [75]:
from pyspark.sql.functions import corr

In [2]:
spark = SparkSession.builder.appName('cruise_ship').getOrCreate()

In [3]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
df.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [7]:
df.count()

158

In [12]:
df.select(['Cruise_line']).distinct().orderBy('Cruise_line').show()

+-----------------+
|      Cruise_line|
+-----------------+
|          Azamara|
|         Carnival|
|        Celebrity|
|            Costa|
|          Crystal|
|           Cunard|
|           Disney|
| Holland_American|
|              MSC|
|        Norwegian|
|          Oceania|
|           Orient|
|              P&O|
|         Princess|
|Regent_Seven_Seas|
|  Royal_Caribbean|
|         Seabourn|
|        Silversea|
|             Star|
|         Windstar|
+-----------------+



In [57]:
df.select('Cruise_line').distinct().count()

20

In [8]:
df.select(['Cruise_line']).groupBy('Cruise_line').count().orderBy('Cruise_line').show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|          Azamara|    2|
|         Carnival|   22|
|        Celebrity|   10|
|            Costa|   11|
|          Crystal|    2|
|           Cunard|    3|
|           Disney|    2|
| Holland_American|   14|
|              MSC|    8|
|        Norwegian|   13|
|          Oceania|    3|
|           Orient|    1|
|              P&O|    6|
|         Princess|   17|
|Regent_Seven_Seas|    5|
|  Royal_Caribbean|   23|
|         Seabourn|    3|
|        Silversea|    4|
|             Star|    6|
|         Windstar|    3|
+-----------------+-----+



In [49]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_cat')
indexed = indexer.fit(df).transform(df)

In [54]:
indexed.select('Cruise_line_cat').describe().show()

+-------+-----------------+
|summary|  Cruise_line_cat|
+-------+-----------------+
|  count|              158|
|   mean|5.063291139240507|
| stddev|4.758744608182735|
|    min|              0.0|
|    max|             19.0|
+-------+-----------------+



In [59]:
df2 = indexed

In [39]:
df.select('crew').describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [10]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [58]:
inputCols = ['Age', 'Tonnage', 'passengers', 'length', 'cabins', 
             'passenger_density', 'Cruise_line_cat']
outputCol = 'features'
assembler = VectorAssembler(inputCols=inputCols, outputCol=outputCol)

In [60]:
output = assembler.transform(df2)

In [61]:
final_data = output.select(['crew', 'features'])

In [62]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [63]:
lr = LinearRegression(labelCol='crew')
type(lr)

pyspark.ml.regression.LinearRegression

In [64]:
lrModel = lr.fit(train_data)

In [65]:
type(lrModel)

pyspark.ml.regression.LinearRegressionModel

In [66]:
lrModel.coefficients

DenseVector([-0.0155, 0.0102, -0.1758, 0.4365, 0.9026, -0.0148, 0.0567])

In [67]:
lrModel.intercept

-0.6466641435688545

In [68]:
test_results = lrModel.evaluate(test_data)

In [69]:
type(test_results)

pyspark.ml.regression.LinearRegressionSummary

In [70]:
test_results.meanSquaredError

0.4580300248170542

In [71]:
test_results.meanAbsoluteError

0.5436836448925243

In [72]:
test_results.explainedVariance

10.98046744845458

In [73]:
test_results.r2

0.9562593926960802

In [74]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.43743726437816477|
| -0.7258419617458335|
| 0.29397977803631603|
|-0.05087327992577961|
| 0.02533745932339837|
| -0.8729111304199533|
| 0.33840453709543716|
|-0.04411063937696991|
| 0.06689723650982815|
| 0.06689723650982815|
| 0.20272037649806673|
|    1.87065237682312|
| 0.05665706236595458|
|-0.11242377328864439|
| 0.12304841690369006|
| -1.3092309506656639|
| 0.18775917403958786|
|-0.14171011067470562|
| -0.6539255342952535|
|  0.5854967492991774|
+--------------------+
only showing top 20 rows



In [42]:
unlabeled_data = test_data.select('features')

In [43]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[12.0,2.329,0.94,...|
|[27.0,10.0,2.08,4...|
|[13.0,25.0,3.82,5...|
|[14.0,30.27699999...|
|[15.0,30.27699999...|
|[10.0,46.0,7.0,6....|
|[21.0,19.093,8.0,...|
|[14.0,63.0,14.4,7...|
|[20.0,53.049,13.4...|
|[27.0,53.872,14.9...|
|[22.0,52.926,13.0...|
|[10.0,68.0,10.8,7...|
|[27.0,53.872,14.9...|
|[16.0,59.652,13.2...|
|[15.0,78.491,24.3...|
|[28.0,46.052,14.5...|
|[26.0,47.262,14.8...|
|[24.0,40.05300000...|
|[17.0,74.137,19.5...|
|[16.0,78.491,24.3...|
+--------------------+
only showing top 20 rows



In [44]:
predictions = lrModel.transform(unlabeled_data)

In [46]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[12.0,2.329,0.94,...|0.4689983384921135|
|[27.0,10.0,2.08,4...|1.3909758305033493|
|[13.0,25.0,3.82,5...|2.9339747516712236|
|[14.0,30.27699999...|3.7145842929831168|
|[15.0,30.27699999...|3.6927350545521054|
|[10.0,46.0,7.0,6....| 2.905048022484259|
|[21.0,19.093,8.0,...|  3.57632311980673|
|[14.0,63.0,14.4,7...| 6.859606893664841|
|[20.0,53.049,13.4...| 6.248657472079767|
|[27.0,53.872,14.9...| 6.881053031378698|
|[22.0,52.926,13.0...|  6.06213212659944|
|[10.0,68.0,10.8,7...| 6.096296153801655|
|[27.0,53.872,14.9...| 7.045631680332798|
|[16.0,59.652,13.2...| 6.477571786215343|
|[15.0,78.491,24.3...|  8.59065309232437|
|[28.0,46.052,14.5...| 6.377030212880024|
|[26.0,47.262,14.8...| 6.483675668951541|
|[24.0,40.05300000...|  6.34168187697631|
|[17.0,74.137,19.5...| 8.956054853563407|
|[16.0,78.491,24.3...| 8.577869312066811|
+--------------------+------------

In [76]:
df2.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_cat']

In [89]:
df2.select(corr(df2['passengers'], df2['crew']).alias('corr_output')).show()

+------------------+
|       corr_output|
+------------------+
|0.9152341306065384|
+------------------+



In [ ]:
df2.select()